In [4]:
import os
import sys
import datetime
import re
import calendar
import requests
import urllib
import pandas
from collections import namedtuple
from sec_edgar_downloader import Downloader

# Get CIKs

In [65]:
url = "https://www.sec.gov/Archives/edgar/cik-lookup-data.txt"
#      https://www.sec.gov/Archives/edgar/cik-lookup-data.txt
inlines = []
print(url)
msg_size = 100000
if False:
  with urllib.request.urlopen(url) as f:
    try:
      lines = f.read().decode('utf-8').split("\n")
    except Exception as e:
      print(e)
print(datetime.datetime.now())
with urllib.request.urlopen(url) as f:
  print(datetime.datetime.now())
  li = 0
  while True:
    try:      
      line = f.readline()
      if not line:
        break
      #data = f.read().decode('utf-8')
      inlines.append(line)
      li += 1
      if li % msg_size == 0:
        print("{0}, {1}".format(li, line))
    except urllib.error.HTTPError as he:
      extype, exval, tb = sys.exc_info()
      pass
      #print("{0}, {1}".format(extype, exval))
    except Exception as e:
      extype, exval, tb = sys.exc_info()
      print("{0}, {1}".format(extype, exval))  
  
print("done reading: {0} lines <{1}>".format(len(lines), datetime.datetime.now()))

https://www.sec.gov/Archives/edgar/cik-lookup-data.txt
2021-05-02 13:33:26.671983
2021-05-02 13:33:26.818779
100000, b'BOYAN BARBARA D:0001283878:\n'
200000, b'DG URBAN-C LP:0001715112:\n'
300000, b'GOLD VALLEY GLOBAL HOLDING CORP.:0001642352:\n'
400000, b'KEYES JEFFRY R.:0001557736:\n'
500000, b'MURRAY DERVAL:0001807413:\n'
600000, b'REBHOLZ DAVID F:0001383752:\n'
700000, b'SYML LYFE, LLC:0001815154:\n'
800000, b'ZIEGLER ANN ELIZABETH:0001214029:\n'
done reading: 801839 lines <2021-05-02 13:33:33.273259>


In [67]:
print(datetime.datetime.now())
cik_file = "cik_file.txt"
msg_size = 100000
bad_lines = []
with open(cik_file, "w") as fp:
  for li, line in enumerate(inlines):
    if len(bad_lines) > 10:
      print("too many bad lines")
      break
    #decoded = False      
    #for decoding in ('utf-8', 'utf-16', "ISO-8859-1"):
    try:
      line_str = str(line)
      line_str = re.sub("b'", '', line_str)             
    except Exception:
      print("ERROR")
      extype, exval, tb = sys.exc_info() 
      print("{0}, {1}".format(extype, exval))                        
      print("{0}, {1}".format(li, line))
      bad_lines.append(line)
      break
    else:
      if decoding != 'utf-8':
        bad_lines.append(line)
        print("OK {0}, {1}\n {2}".format(decoding, li, line_str))         
      decoded = True
      
    try:
      for ch in ["~"]:
        if re.search(ch, line_str):
          print("line {0} has {1}\n {2}".format(li, ch, line))
      parts = line_str.split(":")
      if len(parts) != 3:
        #print("Problem line {0}, len(parts): {1}\n{2}".format(i, len(parts), line))
        #print(",".join(parts))
        pass
      n = len(parts)
      cik = parts[n-2]       
      company = " ".join(parts[:n-2])
      outline = "{0}~{1}\n".format(company, cik)
      fp.writelines(outline)
      
      if li % msg_size == 0:
        print("{0}, {1}".format(li, line))      
        
    except Exception as e:
      print("{0}, {1}".format(li, line))
      extype, exval, tb = sys.exc_info()
      print("{0}, {1}".format(extype, exval))

print("Done {0}".format(datetime.datetime.now()))

2021-05-02 13:34:24.378970
0, b'!J INC:0001438823:\n'
100000, b'BOYAN GEORGE:0001854802:\n'
200000, b'DG VALUE PARNERS LP:0001394325:\n'
300000, b'GOLD VIDEO SHOWCASE LLC:0001422323:\n'
400000, b'KEYES KEVIN:0001508136:\n'
500000, b'MURRAY DONALD B:0001133824:\n'
600000, b'REBHORN BRIAN S.:0001609374:\n'
700000, b'SYMMETRE, INC.:0001752785:\n'
800000, b'ZIEGLER B C & CO                                        /BD:0000109309:\n'
Done 2021-05-02 13:34:37.103687


# Read Daily filing files
## extract 13 F filings

In [84]:
print("Start {0}".format(datetime.datetime.now()))
years = [2021, 2020]
quarters = [1,2,3,4]
pos_markers = [0, 62, 74, 86, 98]
file_fields = ["company", "form", "cik", "date", "url_post"]
CIK = namedtuple("CIK", file_fields)
years = [2021]
keep_dict = {}
for year in years:
  for qtr in quarters:
    months = [m + (qtr-1)*3 for m in [1, 2, 3]]
    url_pre = "https://www.sec.gov/Archives/edgar/daily-index/{0}/QTR{1}/".format(year, qtr)
    for month in months:
      last_day = calendar.monthrange(year, month)[1]      
      for day in range(1, 5): # last_day+1):
        date_str = "{0}{1}{2}".format(str(year), str(month).zfill(2), str(day).zfill(2))

        url_post = "company.{0}.idx".format(date_str)
        url = url_pre  +  url_post
        try:
          with urllib.request.urlopen(url) as f:
            lines = f.read().decode().split("\n")
          print("Got data {0} {1}".format(date_str, datetime.datetime.now()))          
          keep_dict[date_str] = []
          for line in lines:
            if re.search("13", line[pos_markers[1]:pos_merkers[2]]):
              keep_dict[date_str].append(line)
        except urllib.error.HTTPError as he:
          extype, exval, tb = sys.exc_info()
          pass
          #print("{0}, {1}".format(extype, exval))
        except Exception as e:
          extype, exval, tb = sys.exc_info()
          print("{0}, {1}".format(extype, exval))            
print("Done {0}".format(datetime.datetime.now()))

Start 2021-05-02 14:50:38.564639
Got data 20210104 2021-05-02 14:50:39.229814
Got data 20210201 2021-05-02 14:50:39.507413
Got data 20210202 2021-05-02 14:50:39.861483
Got data 20210301 2021-05-02 14:50:40.333402
Got data 20210303 2021-05-02 14:50:40.799664
Got data 20210304 2021-05-02 14:50:41.064765
Got data 20210401 2021-05-02 14:50:41.315940
Got data 20210402 2021-05-02 14:50:41.548228
Done 2021-05-02 14:50:44.564931


In [108]:
msg_size = 500
pos_markers = [0, 62, 74, 86, 98, 0]
tup_dict = {}
for k in keep_dict.keys():
  tup_dict[k] = []    
  print("---{0} {1} lines---".format(k, len(keep_dict[k])))   
  for li, line in enumerate(keep_dict[k]):
    pos_markers[-1] = len(line)
    parts = []
    for i in range(len(pos_markers)-1):
      parts.append(line[pos_markers[i]:pos_markers[i+1]].rstrip())
      #print(i, parts[i]) 
    tup = CIK(*parts)
    tup_dict[k].append(tup)
    if li % msg_size == 0:
      print("{0}, {1}".format(li, tup))  


---20210104 140 lines---
0, CIK(company='ACKRELL SPAC Partners I Co.', form='SC 13D', cik='1790121', date='20210104', url_post='edgar/data/1790121/0001213900-21-000160.txt')
---20210201 1163 lines---
0, CIK(company='1ST SOURCE CORP', form='SC 13G/A', cik='34782', date='20210201', url_post='edgar/data/34782/0001104659-21-010167.txt')
500, CIK(company='EIG VETERAN EQUITY AGGREGATOR, L.P.', form='SC 13D/A', cik='1735742', date='20210201', url_post='edgar/data/1735742/0001193125-21-023541.txt')
1000, CIK(company='Travere Therapeutics, Inc.', form='SC 13G/A', cik='1438533', date='20210201', url_post='edgar/data/1438533/0000834237-21-005445.txt')
---20210202 1285 lines---
0, CIK(company='1 800 FLOWERS COM INC', form='SC 13G/A', cik='1084869', date='20210202', url_post='edgar/data/1084869/0001398344-21-002040.txt')
500, CIK(company='BlackRock Inc.', form='SC 13G', cik='1364742', date='20210202', url_post='edgar/data/1364742/0000834237-21-007135.txt')
1000, CIK(company='Protagonist Therapeutic

In [135]:
for k in tup_dict.keys():
  print("--{0}, {1}--".format(k, len(tup_dict[k])))
  nt = min(len(tup_dict[k]), 0)
  for ti, tup in enumerate(tup_dict[k]):
    print("  ", ti, tup)
    if ti >= nt:
      break
    

--20210104, 140--
   0 CIK(company='ACKRELL SPAC Partners I Co.', form='SC 13D', cik='1790121', date='20210104', url_post='edgar/data/1790121/0001213900-21-000160.txt')
--20210201, 1163--
   0 CIK(company='1ST SOURCE CORP', form='SC 13G/A', cik='34782', date='20210201', url_post='edgar/data/34782/0001104659-21-010167.txt')
--20210202, 1285--
   0 CIK(company='1 800 FLOWERS COM INC', form='SC 13G/A', cik='1084869', date='20210202', url_post='edgar/data/1084869/0001398344-21-002040.txt')
--20210301, 171--
   0 CIK(company='180 DEGREE CAPITAL CORP. /NY/', form='SC 13G', cik='893739', date='20210301', url_post='edgar/data/893739/0000893739-21-000023.txt')
--20210303, 91--
   0 CIK(company='AMH Equity LLC', form='SC 13G/A', cik='1019432', date='20210303', url_post='edgar/data/1019432/0001019432-21-000003.txt')
--20210304, 103--
   0 CIK(company='3i, LP', form='SC 13G', cik='1841619', date='20210304', url_post='edgar/data/1841619/0001104659-21-032212.txt')
--20210401, 146--
   0 CIK(company=

In [139]:

savedir = "./data13f"
if not os.path.isdir(savedir):
  os.makedirs(savedir)
if not os.path.isdir(savedir):
  raise RuntimeError("no dir {0}".format(savedir))
dl = Downloader(savedir)

In [145]:
nt = 100000
for k in tup_dict.keys():
  print("\n--{0}, {1}--\n".format(k, len(tup_dict[k])))
  nt = min(len(tup_dict[k]), 0)
  for ti, tup in enumerate(tup_dict[k]):
    print("  ", ti, tup)
    year = int(tup.date[0:4])
    month = int(tup.date[4:6])
    day = int(tup.date[6:8])
    dt = datetime.date(year, month, day)

    before = (dt + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    after = (dt - datetime.timedelta(days=1)).strftime("%Y-%m-%d")

    try:
      res = dl.get(tup.form, tup.cik, amount=2, after=after, before=before)   
    except urllib.error.HTTPError as he:
      extype, exval, tb = sys.exc_info()
      print("{0}".format(extype, exval))
    except ValueError as e:
      extype, exval, tb = sys.exc_info()
      print("{0}, ".format(extype, exval))       
    except Exception as e:
      extype, exval, tb = sys.exc_info()
      print("{0}, {1}".format(extype, exval)) 
    else:
      print("OK")
      print(res)


--20210104, 140--

   0 CIK(company='ACKRELL SPAC Partners I Co.', form='SC 13D', cik='1790121', date='20210104', url_post='edgar/data/1790121/0001213900-21-000160.txt')
<class 'requests.exceptions.HTTPError'>, 500 Server Error: Internal Server Error for url: https://efts.sec.gov/LATEST/search-index
   1 CIK(company='ACKRELL SPAC Sponsors I LLC', form='SC 13D', cik='1790119', date='20210104', url_post='edgar/data/1790119/0001213900-21-000160.txt')
<class 'requests.exceptions.HTTPError'>, 500 Server Error: Internal Server Error for url: https://efts.sec.gov/LATEST/search-index
   2 CIK(company='ADAGE CAPITAL PARTNERS GP, L.L.C.', form='SC 13G', cik='1165408', date='20210104', url_post='edgar/data/1165408/0000902664-21-000009.txt')
OK
0
   3 CIK(company='ADAGE CAPITAL PARTNERS GP, L.L.C.', form='SC 13G', cik='1165408', date='20210104', url_post='edgar/data/1165408/0000902664-21-000010.txt')
<class 'requests.exceptions.HTTPError'>, 500 Server Error: Internal Server Error for url: https:/

<class 'requests.exceptions.HTTPError'>, 500 Server Error: Internal Server Error for url: https://efts.sec.gov/LATEST/search-index
   44 CIK(company='Eledon Pharmaceuticals, Inc.', form='SC 13G', cik='1404281', date='20210104', url_post='edgar/data/1404281/0000902664-21-000010.txt')
<class 'requests.exceptions.HTTPError'>, 500 Server Error: Internal Server Error for url: https://efts.sec.gov/LATEST/search-index
   45 CIK(company='Eledon Pharmaceuticals, Inc.', form='SC 13G', cik='1404281', date='20210104', url_post='edgar/data/1404281/0000929638-21-000023.txt')
OK
0
   46 CIK(company='Fried Kenneth', form='SC 13G/A', cik='1684822', date='20210104', url_post='edgar/data/1684822/0001104659-21-000171.txt')
<class 'ValueError'>, 
   47 CIK(company='GIC Private Ltd', form='SC 13G', cik='936828', date='20210104', url_post='edgar/data/936828/0001140361-21-000061.txt')
<class 'requests.exceptions.HTTPError'>, 500 Server Error: Internal Server Error for url: https://efts.sec.gov/LATEST/search-i

<class 'requests.exceptions.HTTPError'>, 500 Server Error: Internal Server Error for url: https://efts.sec.gov/LATEST/search-index
   91 CIK(company='Parfield International Ltd', form='SC 13D/A', cik='1657085', date='20210104', url_post='edgar/data/1657085/0000898432-21-000001.txt')
<class 'ValueError'>, 
   92 CIK(company='Pensionfund DSM Netherlands', form='13F-HR', cik='1297731', date='20210104', url_post='edgar/data/1297731/0001297731-21-000001.txt')
OK
0
   93 CIK(company='Pensionfund Sabic', form='13F-HR', cik='1637246', date='20210104', url_post='edgar/data/1637246/0001637246-21-000001.txt')
<class 'requests.exceptions.HTTPError'>, 500 Server Error: Internal Server Error for url: https://efts.sec.gov/LATEST/search-index
   94 CIK(company='Porch Group, Inc.', form='SC 13G/A', cik='1784535', date='20210104', url_post='edgar/data/1784535/0001213900-21-000196.txt')
<class 'ValueError'>, 
   95 CIK(company='ROCKLAND TRUST CO', form='13F-HR', cik='84616', date='20210104', url_post='ed

OK
0
   132 CIK(company='Watford Holdings Ltd.', form='SC 13E3', cik='1601669', date='20210104', url_post='edgar/data/1601669/0001140361-21-000020.txt')
OK
0
   133 CIK(company='Willner Morris', form='SC 13D', cik='1837162', date='20210104', url_post='edgar/data/1837162/0001193125-21-001026.txt')
<class 'requests.exceptions.HTTPError'>, 500 Server Error: Internal Server Error for url: https://efts.sec.gov/LATEST/search-index
   134 CIK(company='Wright Investors Service Holdings, Inc.', form='SC 13D/A', cik='1279715', date='20210104', url_post='edgar/data/1279715/0001085146-21-000002.txt')
<class 'ValueError'>, 
   135 CIK(company='Wright Investors Service Holdings, Inc.', form='SC 13D/A', cik='1279715', date='20210104', url_post='edgar/data/1279715/0001214659-21-000038.txt')
<class 'ValueError'>, 
   136 CIK(company='XPEL, Inc.', form='SC 13G/A', cik='1767258', date='20210104', url_post='edgar/data/1767258/0001214659-21-000006.txt')
<class 'ValueError'>, 
   137 CIK(company='YUMANITY T

OK
0
   38 CIK(company='Atlas Venture Advisors, Inc.', form='13F-HR', cik='1767432', date='20210201', url_post='edgar/data/1767432/0001104659-21-010103.txt')
OK
0
   39 CIK(company='Atlas Venture Associates VI, Inc.', form='13F-NT', cik='1292360', date='20210201', url_post='edgar/data/1292360/0001104659-21-010105.txt')
<class 'requests.exceptions.HTTPError'>, 500 Server Error: Internal Server Error for url: https://efts.sec.gov/LATEST/search-index
   40 CIK(company='Atlas Venture Associates VI, L.P.', form='13F-NT', cik='1292359', date='20210201', url_post='edgar/data/1292359/0001104659-21-010099.txt')
<class 'requests.exceptions.HTTPError'>, 500 Server Error: Internal Server Error for url: https://efts.sec.gov/LATEST/search-index
   41 CIK(company='Atlas Venture Associates XI, LLC', form='13F-HR', cik='1780067', date='20210201', url_post='edgar/data/1780067/0001104659-21-010104.txt')
OK
0
   42 CIK(company='Ault Global Holdings, Inc.', form='SC 13D', cik='896493', date='20210201', url

OK
0
   86 CIK(company='BHP Group Plc', form='SC 13G/A', cik='1171264', date='20210201', url_post='edgar/data/1171264/0001374170-21-000013.txt')
<class 'ValueError'>, 
   87 CIK(company='BICYCLE THERAPEUTICS plc', form='SC 13G/A', cik='1761612', date='20210201', url_post='edgar/data/1761612/0001104659-21-010302.txt')
<class 'ValueError'>, 
   88 CIK(company='BIGGER CAPITAL FUND L P', form='SC 13G/A', cik='1288478', date='20210201', url_post='edgar/data/1288478/0000921895-21-000197.txt')
<class 'ValueError'>, 
   89 CIK(company='BIOTECHNOLOGY VALUE FUND L P', form='SC 13G', cik='918923', date='20210201', url_post='edgar/data/918923/0000921895-21-000198.txt')
<class 'requests.exceptions.HTTPError'>, 500 Server Error: Internal Server Error for url: https://efts.sec.gov/LATEST/search-index
   90 CIK(company='BNY MELLON HIGH YIELD STRATEGIES FUND', form='SC 13G/A', cik='1057861', date='20210201', url_post='edgar/data/1057861/0001445546-21-000628.txt')
<class 'ValueError'>, 
   91 CIK(compan

OK
0
   128 CIK(company='BlackRock Inc.', form='SC 13G/A', cik='1364742', date='20210201', url_post='edgar/data/1364742/0000834237-21-005441.txt')
<class 'ValueError'>, 
   129 CIK(company='BlackRock Inc.', form='SC 13G/A', cik='1364742', date='20210201', url_post='edgar/data/1364742/0000834237-21-005442.txt')
<class 'ValueError'>, 
   130 CIK(company='BlackRock Inc.', form='SC 13G/A', cik='1364742', date='20210201', url_post='edgar/data/1364742/0000834237-21-005443.txt')
<class 'ValueError'>, 
   131 CIK(company='BlackRock Inc.', form='SC 13G/A', cik='1364742', date='20210201', url_post='edgar/data/1364742/0000834237-21-005444.txt')
<class 'ValueError'>, 
   132 CIK(company='BlackRock Inc.', form='SC 13G/A', cik='1364742', date='20210201', url_post='edgar/data/1364742/0000834237-21-005445.txt')
<class 'ValueError'>, 
   133 CIK(company='BlackRock Inc.', form='SC 13G/A', cik='1364742', date='20210201', url_post='edgar/data/1364742/0000834237-21-005446.txt')
<class 'ValueError'>, 
   13

OK
0
   429 CIK(company='Brand Asset Management Group, Inc.', form='13F-HR', cik='1748766', date='20210201', url_post='edgar/data/1748766/0001104659-21-010048.txt')


KeyboardInterrupt: 